In [41]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import geopandas as gpd

import folium
from folium import Choropleth,Marker
from folium.plugins import HeatMap,MarkerCluster

import shapely
from shapely.geometry import Point, LineString, Polygon, MultiPoint, MultiLineString, MultiPolygon

from datetime import datetime, timedelta

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

In [4]:
data = pd.read_csv("../input/us-accidents/US_Accidents_Dec20_updated.csv",parse_dates = ["Start_Time","End_Time"])
display(data)

In [5]:
data.dtypes

In [6]:
SF_data = data[data["County"] == "San Francisco"]
SF_data.head()

In [7]:
print(SF_data.Start_Lat.max(), SF_data.Start_Lng.max())
print(SF_data.Start_Lat.min(), SF_data.Start_Lng.min())

In [23]:
def convert_to_hours(delta):
    total_seconds = delta.total_seconds()
    hours = str(int(total_seconds // 3600)).zfill(2)
    minutes = str(int((total_seconds % 3600) // 60)).zfill(2)
    seconds = str(int(total_seconds % 60)).zfill(2)
    return f"{hours}:{minutes}:{seconds}"

SF_data["accident_duration"] = SF_data["End_Time"] - SF_data["Start_Time"]


SF_data[SF_data["accident_duration"]  < "0 days 01:00:00"]

In [10]:
m_2 = folium.Map(location=[37.75,-122.38], zoom_start=12)

for idx, row in SF_data[(SF_data["Start_Time"].dt.year == 2019) & (SF_data["Start_Time"].dt.month == 6)].iterrows():
    Marker([row['Start_Lat'], row['Start_Lng']]).add_to(m_2)

embed_map(m_2, 'q_2.html')

In [11]:
m_3 = folium.Map(location=[37.75,-122.38], zoom_start=12)

HeatMap(data=SF_data[['Start_Lat', 'Start_Lng']], radius=10).add_to(m_3)

embed_map(m_3, 'q_3.html')

In [12]:
SF_data.Severity.value_counts()

In [31]:
def color_producer(val):
    if val == 1:
        return 'forestgreen'
    elif val == 2:
        return 'darkbleue'
    elif val == 3:
        return 'darkorange'
    else : 
        return 'darkred'


def color_producer2(val):
    if val < timedelta(hours=1, minutes=0, seconds=0):
        return 'forestgreen'
    elif val < timedelta(hours=3, minutes=0, seconds=0):
        return 'darkorange'
    else : 
        return 'darkred'

In [27]:

m_4 = folium.Map(location=[37.75,-122.38], zoom_start=12)

for i in range(0,len(SF_data)):
    folium.Circle(
        location=[SF_data.iloc[i]['Start_Lat'], SF_data.iloc[i]['Start_Lng']],
        color=color_producer(SF_data.iloc[i]['Severity']),
        radius=SF_data.iloc[i]['Distance(mi)']**8).add_to(m_4)
    
embed_map(m_4, 'q_4.html')

In [35]:
m_5 = folium.Map(location=[37.75,-122.38], zoom_start=12)

for i in range(0,len(SF_data)):
    folium.Circle(
        location=[SF_data.iloc[i]['Start_Lat'], SF_data.iloc[i]['Start_Lng']],
        color=color_producer2(SF_data.iloc[i]['accident_duration']),
        radius=SF_data.iloc[i]['Distance(mi)']**4).add_to(m_5)
    
embed_map(m_5, 'q_5.html')

In [39]:
Geo_df = gpd.GeoDataFrame(SF_data, geometry=gpd.points_from_xy(SF_data.Start_Lat, SF_data.Start_Lng))
display(Geo_df)

In [ ]:
geometry = [Point(xy) for xy in zip(SF_data.X, df.Y)]
geo_df = GeoDataFrame(df, geometry=geometry)

In [46]:
#geo_df["line"] = LineString([(SF_data.Start_Lat,SF_data.Start_Lng),(SF_data.End_Lat, SF_data.End_Lng)])


In [45]:
line = LineString([(0,0),(1,2)])
line